In [51]:
!pip install ipython-sql

In [52]:
!pip install pymysql

In [53]:
# now we ant to load the sql extension after installing mysql to colab
%load_ext sql


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [54]:
#source for data: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce?resource=download

In [55]:
#converting a csv file into sql tables
import sqlite3
import pandas as pd

For the first section, we'll want to convert a csv file into useable data for sql queries

In [56]:
# read the csv file and clean the data
df = pd.read_csv('olist_customers_dataset.csv')
#for any string data in the columns, we'll be stripping empty spaces
df.columns = df.columns.str.strip()

In [57]:
#create and connect to the sqlite database
connection = sqlite3.connect('Practice.db')
#now load the data to sqlite: df.to_sql('table_name', connection, if_exists='__') <- if_exists is used if the table name already exists, where we can then make appropriate cahnges in the form of fail, replace, or append

df.to_sql('Olist_Ecommerce_Data', connection, if_exists='replace')

99441

In [58]:
#we need to close the connection after loading the data
connection.close()

In [59]:
#now we want to call and read that database
connection = sqlite3.connect('Practice.db')
%sql sqlite:///Practice.db



# **To run sql queries, we need to initialize it with  %%sql before going through the actual code**

In [60]:
# This is how we'll call it with sqlite where we initialize it w/ %%sql
# %%sql
# SELECT customer_state, COUNT(customer_state) AS total
# FROM Olist_Ecommerce_Data
# GROUP BY customer_state
# ORDER BY customer_state ASC;

In [61]:
## How many purchases were from a particular state?
query = """
SELECT customer_state, COUNT(customer_state) AS total
FROM Olist_Ecommerce_Data
GROUP BY customer_state
ORDER BY customer_state ASC;
"""
df_result = pd.read_sql_query(query, connection)
df_result

,customer_state,total
0,AC,81
1,AL,413
2,AM,148
3,AP,68
4,BA,3380
5,CE,1336
6,DF,2140
7,ES,2033
8,GO,2020
9,MA,747


In [62]:
#To better seethe data
query2 = """
SELECT *
FROM Olist_Ecommerce_Data
"""
df_result2 = pd.read_sql_query(query2, connection)
df_result2

,index,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...,...
99436,99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [63]:
# Finding the number of orders per city in the State, SP
query3 = """
SELECT customer_city, COUNT(customer_city) as totalNum
FROM Olist_Ecommerce_Data
WHERE customer_state = "SP"
GROUP BY customer_city
ORDER BY customer_state ASC
"""
df_result3 = pd.read_sql(query3, connection)
df_result3

,customer_city,totalNum
0,adamantina,36
1,adolfo,5
2,agisse,1
3,aguai,10
4,aguas da prata,10
...,...,...
624,viradouro,18
625,vista alegre do alto,3
626,votorantim,85
627,votuporanga,63


In [64]:
query4 = """
SELECT customer_state, customer_zip_code_prefix
FROM Olist_Ecommerce_Data
WHERE customer_zip_code_prefix between "10000" and "50000"
GROUP BY customer_zip_code_prefix
ORDER BY customer_state ASC, customer_zip_code_prefix DESC
"""
df_results4 = pd.read_sql_query(query4, connection)
df_results4

,customer_state,customer_zip_code_prefix
0,BA,48990
1,BA,48970
2,BA,48967
3,BA,48950
4,BA,48930
...,...,...
5354,SP,11025
5355,SP,11020
5356,SP,11015
5357,SP,11013


In [65]:
connection.close()

# **For the second section, we'll be understanding sales and the movement of products**

In [66]:
conn = sqlite3.connect('AdventureWorks.db')
%sql sqlite:///AdventureWorks.db


In [67]:
#We want to have a better understanding of the data and the relational database tables that we're working with
#We'll be using 3 of the tables: SalesOrderDetail, Product, and SalesOrderHeader

In [68]:
query = """
SELECT *
FROM SalesOrderDetail
LIMIT 5

"""
df_r = pd.read_sql_query(query, conn)
df_r

,salesorderid,salesorderdetailid,carriertrackingnumber,orderqty,productid,specialofferid,unitprice,unitpricediscount,rowguid,modifieddate
0,43659,1,4911-403C-98,1,776,1,2024.994,0.0,b207c96d-d9e6-402b-8470-2cc176c42283,2011-05-31 00:00:00
1,43659,2,4911-403C-98,3,777,1,2024.994,0.0,7abb600d-1e77-41be-9fe5-b9142cfc08fa,2011-05-31 00:00:00
2,43659,3,4911-403C-98,1,778,1,2024.994,0.0,475cf8c6-49f6-486e-b0ad-afc6a50cdd2f,2011-05-31 00:00:00
3,43659,4,4911-403C-98,1,771,1,2039.994,0.0,04c4de91-5815-45d6-8670-f462719fbce3,2011-05-31 00:00:00
4,43659,5,4911-403C-98,1,772,1,2039.994,0.0,5a74c7d2-e641-438e-a7ac-37bf23280301,2011-05-31 00:00:00


In [69]:
query2 = """
SELECT *
FROM SalesOrderHeader
LIMIT 5
"""
df_r2 = pd.read_sql_query(query2, conn)
df_r2

,salesorderid,revisionnumber,orderdate,duedate,shipdate,STATUS,onlineorderflag,purchaseordernumber,accountnumber,customerid,...,creditcardid,creditcardapprovalcode,currencyrateid,subtotal,taxamt,freight,totaldue,comment,rowguid,modifieddate
0,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO522145787,10-4020-000676,29825,...,16281,105041Vi84182,,20565.6206,1971.5149,616.0984,23153.2339,,79b65321-39ca-4115-9cba-8fe0903e12e6,2011-06-07 00:00:00
1,43660,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO18850127500,10-4020-000117,29672,...,5618,115213Vi29411,,1294.2529,124.2483,38.8276,1457.3288,,738dc42d-d03b-48a1-9822-f95a67ea7389,2011-06-07 00:00:00
2,43661,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO18473189620,10-4020-000442,29734,...,1346,85274Vi6854,4,32726.4786,3153.7696,985.5530,36865.8012,,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07 00:00:00
3,43662,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO18444174044,10-4020-000227,29994,...,10456,125295Vi53935,4,28832.5289,2775.1646,867.2389,32474.9324,,4a1ecfc0-cc3a-4740-b028-1c50bb48711c,2011-06-07 00:00:00
4,43663,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO18009186470,10-4020-000510,29565,...,4322,45303Vi22691,,419.4589,40.2681,12.5838,472.3108,,9b1e7a40-6ae0-4ad3-811c-a64951857c4b,2011-06-07 00:00:00


In [98]:
query3 = """
SELECT *
FROM Product
LIMIT 5
"""
df_r3 = pd.read_sql_query(query3, conn)
df_r3

,productid,NAME,productnumber,makeflag,finishedgoodsflag,color,safetystocklevel,reorderpoint,standardcost,listprice,...,productline,class,style,productsubcategoryid,productmodelid,sellstartdate,sellenddate,discontinueddate,rowguid,modifieddate
0,1,Adjustable Race,AR-5381,f,f,,1000,750,0.0,0.0,...,,,,,,2008-04-30 00:00:00,,,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,f,f,,1000,750,0.0,0.0,...,,,,,,2008-04-30 00:00:00,,,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,t,f,,800,600,0.0,0.0,...,,,,,,2008-04-30 00:00:00,,,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,f,f,,800,600,0.0,0.0,...,,,,,,2008-04-30 00:00:00,,,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,t,f,,800,600,0.0,0.0,...,,,,,,2008-04-30 00:00:00,,,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827


In [71]:
#Calculate the total revenue from all the transactions in this dataset
query5 = """
SELECT SUM(s.orderqty * s.unitprice) as Total_Revenue
FROM SalesOrderDetail s
"""
result = pd.read_sql_query(query5, conn)
result

,Total_Revenue
0,1.103739e+08


In [83]:
# Calculate the number of orders for a product and the rev generated for the month of May, then compare it to the average
query6 = """
SELECT
    p.NAME,
    strftime('%Y', h.orderdate) AS Order_Year,
    SUM(s.orderqty) AS Number_of_orders,
    SUM(s.orderqty * s.unitprice) AS Revenue_by_product,
    (
        SELECT
            SUM(x.orderqty * x.unitprice)
        FROM
            SalesOrderDetail x
        JOIN
            SalesOrderHeader y ON x.salesorderid = y.salesorderid
        WHERE
            strftime('%m', y.orderdate) = '05'
            AND strftime('%Y', y.orderdate) = strftime('%Y', h.orderdate)
    ) AS May_Revenue
FROM
    SalesOrderDetail s, Product p, SalesOrderHeader h
WHERE
  s.productid = p.productid AND s.salesorderid = h.salesorderid AND strftime('%m', h.orderdate) = '05'
GROUP BY
    p.NAME, Order_Year
ORDER BY
    p.NAME ASC, Order_Year ASC;

"""
result2 = pd.read_sql_query(query6, conn)
result2

,NAME,Order_Year,Number_of_orders,Revenue_by_product,May_Revenue
0,AWC Logo Cap,2011,40,207.4600,5.038059e+05
1,AWC Logo Cap,2012,215,1093.6594,3.092482e+06
2,AWC Logo Cap,2013,261,1337.5322,3.309156e+06
3,AWC Logo Cap,2014,456,3302.0270,5.379614e+06
4,All-Purpose Bike Stand,2013,1,159.0000,3.309156e+06
...,...,...,...,...,...
524,"Women's Mountain Shorts, S",2013,178,6852.0210,3.309156e+06
525,"Women's Mountain Shorts, S",2014,294,12838.9656,5.379614e+06
526,"Women's Tights, L",2012,189,8263.8980,3.092482e+06
527,"Women's Tights, M",2012,34,1529.7960,3.092482e+06


In [73]:
#Determine the number of purcahses based on a given day of the week
query7 = """
SELECT
   CAST(strftime('%w', h.orderdate) as INT) AS Day_of_the_Week, SUM(s.orderqty) AS Total_Purchases, AVG(s.orderqty) as Average_Purchases
FROM
    SalesOrderDetail s, SalesOrderHeader h
WHERE
    s.salesorderid = h.salesorderid
GROUP BY
    Day_of_the_Week
"""
result3 = pd.read_sql_query(query7, conn)
result3

,Day_of_the_Week,Total_Purchases,Average_Purchases
0,0,35771,2.185022
1,1,44649,2.255114
2,2,37121,2.231097
3,3,50546,2.496222
4,4,40248,2.377740
5,5,24263,1.840476
6,6,42316,2.331460


In [99]:
query8 ="""
SELECT
    p.NAME, p.productid, SUM(s.orderqty) as Number_Ordered
FROM
    SalesOrderDetail s
JOIN
    Product p on s.productid = p.productid
GROUP BY
    p.NAME
HAVING
    Number_Ordered > 100
ORDER BY
    Number_Ordered DESC
"""
result4 = pd.read_sql_query(query8, conn)
result4

,NAME,productid,Number_Ordered
0,AWC Logo Cap,712,8311
1,Water Bottle - 30 oz.,870,6815
2,"Sport-100 Helmet, Blue",711,6743
3,"Long-Sleeve Logo Jersey, L",715,6592
4,"Sport-100 Helmet, Black",708,6532
...,...,...,...
238,"LL Road Frame - Black, 60",723,129
239,"HL Touring Frame - Blue, 50",891,124
240,"HL Touring Frame - Blue, 46",890,114
241,"HL Road Frame - Red, 48",719,109


In [101]:
conn.close()